In [5]:
import cv2
import numpy as np
import time

In [ ]:
import os
import pathlib
import sys
from pathlib import __file__

sys.path.insert(0, pathlib.Path().absolute().parent.parent.parent.__str__())
sys.path.append(os.path.join(os.path.dirname(__file__), "../../../../src"))

sys.dont_write_bytecode = True

from utils.holistic.holistic_detector import HolisticDetector

In [ ]:
FPS = 20
DURATION = 5

signs = np.array(["a veces", "abandonar", "abrazar", "abrigo", "abril", "aceptar"])
duration = FPS * DURATION

start_folder = 1
cant_videos_per_sign = 30

In [ ]:
ptiempo = 0

# Crearemos el objeto
holistic = HolisticDetector()

# Leemos la cámara web
cap = cv2.VideoCapture(0)

for sign in signs:
    for video_num in range(start_folder, start_folder+cant_videos_per_sign):
        result_of_video = []
        for frame_num in range(duration):

            # Leemos el fotograma de la cámara
            success, image = cap.read()

            if not success:
                break
            
            # Una vez que obtengamos la imagen la enviaremos
            results = holistic.detect_holistic(image)
            
            # Dibujamos la predicción en el fotograma
            image = holistic.draw_prediction(image, results)

            # Mostramos los fps
            ctiempo = time.time()
            fps = 1 / (ctiempo - ptiempo)
            ptiempo = ctiempo
            cv2.putText(image, str(int(fps)), (image.shape[1]-100, 70), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3)

            if frame_num == 0: 
                cv2.putText(image, 'STARTING COLLECTION', (120,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(sign, video_num), (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                # Show to screen
                cv2.imshow('OpenCV Feed', image)
                cv2.waitKey(500)
            else: 
                cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(sign, video_num), (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                # Show to screen
                cv2.imshow('OpenCV Feed', image)

            pose, right_hand, left_hand, face = holistic.get_unprocessed_coordenates(results)
            coords = [pose, right_hand, left_hand, face]
            result_of_video.append(coords)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

cap.release()
cv2.destroyAllWindows()


In [9]:
import cv2
ptiempo = 0
cap= cv2.VideoCapture(0)

width= int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height= int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

writer= cv2.VideoWriter('basicvideo.mp4', cv2.VideoWriter_fourcc(*'DIVX'), 20, (width,height))


while True:
    ret,frame= cap.read()


    ctiempo = time.time()
    fps = 1 / (ctiempo - ptiempo)
    ptiempo = ctiempo
    cv2.putText(frame, str(int(fps)), (frame.shape[1]-100, 70), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3)

    writer.write(frame)

    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break


cap.release()
writer.release()
cv2.destroyAllWindows()


In [ ]:
import time

import cv2
from utils.holistic.holistic_detector import HolisticDetector

ptiempo = 0

# Crearemos el objeto
holistic = HolisticDetector()

# Leemos la cámara web
cap = cv2.VideoCapture(0)

while cap.isOpened():
    
    # Leemos el fotograma de la cámara
    success, image = cap.read()

    if not success:
        break

    # Una vez que obtengamos la imagen la enviaremos
    results = holistic.detect_holistic(image)
    
    # Dibujamos la predicción en el fotograma
    image = holistic.draw_prediction(image, results)

    # Predicción
    x = holistic.get_coordenates(results, used_parts=["pose", "right_hand", "left_hand"])
    try:
        res = modelo.get_prediction(x, classes)
    except Exception as e:
        print("Error: ", e)

    # Viz probabilities
    image = holistic.probability_visualizer(res, classes, image)

    # Mostramos los fps
    ctiempo = time.time()
    fps = 1 / (ctiempo - ptiempo)
    ptiempo = ctiempo
    cv2.putText(image, str(int(fps)), (image.shape[1]-100, 70), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3)

    # Mostramos todo el fotograma modificado
    cv2.imshow("holistic", image)

    # Break gracefully
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

    
cap.release()
cv2.destroyAllWindows()